# Mixtures of MNLs

In [41]:
import gurobipy as gp

In [42]:
import numpy as np
import numpy.random as npr
N = 3 #5 #25 50 75 100
M = 1 #1 5 10 20
v = np.zeros((N,M))
for i in range(N):
    for j in range(M):
        v[i,j] = max(round(npr.randint(0, 2) + npr.rand()*0.1, 2), 0.01)
theta = list(np.random.dirichlet(np.ones(M), size=1)[0])
print(v)
print(theta)

[[1.01]
 [0.01]
 [1.01]]
[1.0]


In [43]:
def IP_mmnl(p):
    global v, N
    global theta, M

    #create a new model
    model = gp.Model("IP_mmnl")

    B = 10*max(p)
    # create decision variables and store them in the arrays z, x, y
    z = model.addVars(M, vtype=gp.GRB.CONTINUOUS, lb =-np.infty, ub = B, name="z")
    x = model.addVars(N, M, vtype=gp.GRB.CONTINUOUS, lb=-np.infty, ub = B, name="x")
    y = model.addVars(N, vtype=gp.GRB.BINARY, name="y")
    model.update()

    # set objective function
    objExpr = gp.LinExpr()
    for j in range(M):
        objExpr += theta[j] * z[j]
    model.setObjective(objExpr, gp.GRB.MAXIMIZE)
    model.update()

    # add constraints
    for j in range(M):
        myConstr = gp.LinExpr()
        for i in range(N):
            myConstr += v[i][j] * x[i, j]
            model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = y[i]*B, name = "xa_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.GREATER_EQUAL, rhs = -y[i]*B, name = "xb_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.GREATER_EQUAL, rhs = p[i] - z[j] - (1-y[i])*B, name = "xc_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = p[i] - z[j] + (1-y[i])*B, name = "xd_" + str(i) + '_' + str(j))
        model.addConstr(lhs = z[j], sense = gp.GRB.LESS_EQUAL, rhs = myConstr, name = "z_" + str(j))
    model.update()

    # write model 
    model.write("IP_mmnl.lp")

    # solve model
    model.Params.OutputFlag = 0
    model.optimize()
    
    bestS = []
    profit = float(model.objVal)
    for i in range(N):
        if y[i].x > 0.5:
            bestS.append(i)
    bestS = sorted(bestS)
    return profit, bestS

In [44]:
def profit_mmnl(p, S):
    global v
    global theta, M
    pi = 0
    for j in range(M):
        Vj_S = 0
        num = 0
        for i in S:
            num += p[i]*v[i][j]
            Vj_S += v[i][j]
        pi += theta[j]*num/(1+Vj_S)
    return pi

In [45]:
def brute_force(p):
    global N
    # compute the profit of each assortment
    best_profit = 0
    best_S = []
    for i in range(2**N):
        binary = np.binary_repr(i, width=N)
        assortment = [int(x) for x in binary]
        S = []
        for j in range(N):
            if assortment[j] == 1:
                S.append(j)
        if best_profit < profit_mmnl(p, S):
            best_profit = profit_mmnl(p, S)
            best_S = S
    return best_profit, best_S

In [46]:
def nested_by_price_mmnl(p):
    global N
    profits = [0 for i in range(N)]
    S = []
    for i in range(N):
        S.append(i)
        profits[i] = profit_mmnl(p, S)
    best_S = [i for i in range(np.argmax(profits)+1)]
    return max(profits), best_S

In [47]:
def find_best_option_mmnl(p, S):
    global v, N
    current_best = profit_mmnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mmnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mmnl(p, newS)
    return current_best, bestS, flag

def greedy_mmnl(p):
    global v, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mmnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [48]:
import matplotlib.pyplot as plt
import math
import statistics as st
import time

In [49]:
n_simulations = 1000
avg_price = 10
std_price = 20
#compute prices
psim = [0 for k in range(n_simulations)]
for k in range(n_simulations):
    psim[k] = abs(npr.normal(avg_price, std_price, N))
    # p[i] = npr.uniform(0, 2*avg_price, N)
    # p[i] = npr.exponential(avg_price, N)
    psim[k] = np.round(psim[k], 2)
    psim[k] = sorted(psim[k], reverse=True)

In [50]:
# problem solved using the MIP formulation
profits_IP = []
times_IP = []
bestS_IP = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = IP_mmnl(psim[k])
    end_time = time.time()
    profits_IP.append(profit)
    times_IP.append(end_time-start_time)
    bestS_IP.append(bestS)
avg_time_IP = st.mean(times_IP)

In [51]:
# problem solved using the nested formulation
profits_nested = []
times_nested = []
bestS_nested = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = nested_by_price_mmnl(psim[k])
    end_time = time.time()
    profits_nested.append(profit)
    times_nested.append(end_time-start_time)
    bestS_nested.append(bestS)
avg_time_nested = st.mean(times_nested)

In [52]:
# problem solved using the greedy formulation
profits_greedy = []
times_greedy = []
bestS_greedy = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = greedy_mmnl(psim[k])
    end_time = time.time()
    profits_greedy.append(profit)
    times_greedy.append(end_time-start_time)
    bestS_greedy.append(bestS)
avg_time_greedy = st.mean(times_greedy)

In [53]:
# counting how many instances are different
count_nested = 0
different_results_nested = []
count_greedy = 0
different_results_greedy = []
for k in range(n_simulations):
    if bestS_IP[k] != bestS_greedy[k]:
        count_greedy += 100/n_simulations
        different_results_greedy.append(100*profits_greedy[k]/profits_IP[k])
        print(k, bestS_IP[k], bestS_greedy[k], bestS_nested[k])
    if bestS_IP[k] != bestS_nested[k]:
        count_nested += 100/n_simulations
        different_results_nested.append(100*profits_nested[k]/profits_IP[k])
        print(k, bestS_IP[k], bestS_greedy[k], bestS_nested[k])

87 [0] [0, 1] [0, 1]
87 [0] [0, 1] [0, 1]
148 [0] [0, 1] [0, 1]
148 [0] [0, 1] [0, 1]
157 [0] [0, 1] [0, 1]
157 [0] [0, 1] [0, 1]
183 [0] [0, 1] [0, 1]
183 [0] [0, 1] [0, 1]
267 [0] [0, 1] [0, 1]
267 [0] [0, 1] [0, 1]
526 [0] [0, 1] [0, 1]
526 [0] [0, 1] [0, 1]
719 [0] [0, 1] [0, 1]
719 [0] [0, 1] [0, 1]
789 [0] [0, 1] [0, 1]
789 [0] [0, 1] [0, 1]
811 [0] [0, 1] [0, 1]
811 [0] [0, 1] [0, 1]
864 [0] [0, 1] [0, 1]
864 [0] [0, 1] [0, 1]
910 [0, 2] [0, 1, 2] [0, 1, 2]
910 [0, 2] [0, 1, 2] [0, 1, 2]


In [54]:
if different_results_greedy != []:
  print('The greedy algorithm is different from the MIP in ' + str(count_greedy) + ' of the instances' , \
        'with an average ratio of ' + str(st.mean(different_results_greedy)) + '%', \
              'and a minimum ratio of ' + str(min(different_results_greedy)) + '%', \
                'and an average time of ' + str(avg_time_greedy) + ' seconds')
if different_results_nested != []:
  print('The nested algorithm is different from the MIP in ' + str(count_nested) + ' of the instances', \
        'with an average ratio of ' + str(st.mean(different_results_nested)) + '%', \
              'and a minimum ratio of ' + str(min(different_results_nested)) + '%', 
                'and an average time of ' + str(avg_time_nested) + ' seconds')

The greedy algorithm is different from the MIP in 1.0999999999999999 of the instances with an average ratio of 100.00621913400529% and a minimum ratio of 100.000474430689% and an average time of 2.7790546417236328e-05 seconds
The nested algorithm is different from the MIP in 1.0999999999999999 of the instances with an average ratio of 100.00621913400528% and a minimum ratio of 100.000474430689% and an average time of 2.118706703186035e-05 seconds


In [62]:
brute_force(psim[87])


(18.66905940594059, [0, 1])

In [70]:
p = psim[87]
#create a new model
model = gp.Model("IP_mmnl")

B = 10*max(p)
# create decision variables and store them in the arrays z, x, y
z = model.addVars(M, vtype=gp.GRB.CONTINUOUS, lb =-np.infty, ub = B, name="z")
x = model.addVars(N, M, vtype=gp.GRB.CONTINUOUS, lb=-np.infty, ub = B, name="x")
y = model.addVars(N, vtype=gp.GRB.BINARY, name="y")
model.update()

# set objective function
objExpr = gp.LinExpr()
for j in range(M):
    objExpr += theta[j] * z[j]
model.setObjective(objExpr, gp.GRB.MAXIMIZE)
model.update()

# add constraints
# model.addConstr(lhs = y[1], sense = gp.GRB.EQUAL, rhs = 1, name = 'y')
for j in range(M):
    myConstr = gp.LinExpr()
    for i in range(N):
        myConstr += v[i][j] * x[i, j]
        model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = y[i]*B, name = "xa_" + str(i) + '_' + str(j))
        model.addConstr(lhs = x[i, j], sense = gp.GRB.GREATER_EQUAL, rhs = -y[i]*B, name = "xb_" + str(i) + '_' + str(j))
        model.addConstr(lhs = x[i, j], sense = gp.GRB.GREATER_EQUAL, rhs = p[i] - z[j] - (1-y[i])*B, name = "xc_" + str(i) + '_' + str(j))
        model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = p[i] - z[j] + (1-y[i])*B, name = "xd_" + str(i) + '_' + str(j))
    model.addConstr(lhs = z[j], sense = gp.GRB.EQUAL, rhs = myConstr, name = "z_" + str(j))
model.update()

# write model 
model.write("IP_mmnl.lp")

# solve model
# model.Params.OutputFlag = 0
model.optimize()

# print solution
if model.status == gp.GRB.OPTIMAL:
    print('Optimal objective: %g' % model.objVal)
    for j in range(M):
        print('z[' + str(j) + '] = %g' % z[j].x)
    for i in range(N):
        print('y[' + str(i) + '] = %g' % y[i].x)
    for i in range(N):
        for j in range(M):
            print('x[' + str(i) + ',' + str(j) + '] = %g' % x[i, j].x)
else:
    print('No solution')
print(p)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 13 rows, 7 columns and 34 nonzeros
Model fingerprint: 0x9dcd672d
Variable types: 4 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 4e+02]
  RHS range        [3e+02, 4e+02]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 13 rows, 7 columns, 34 nonzeros
Variable types: 4 continuous, 3 integer (3 binary)

Root relaxation: objective 1.880219e+02, 10 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   44.15812    0    3   -0.00000   44.15812      -     -    0s
H    0     0                       3.4269652   44.15812  1189%     -    0s
H    0     0                      18.6674129   44.15812   137%     -    0s
     0     0   18.66906    0    3   18